In [1]:
import spacy
import textacy 

In [2]:
import pandas as pd
import numpy as np 
import matplotlib

%matplotlib inline

In [3]:
df_train = pd.read_csv('data/train_clean_vB.csv', 
                       dtype={
                           'q1_clean_vB': unicode,
                           'q2_clean_vB': unicode
                       }, encoding='utf-8')



df_test = pd.read_csv('data/test_clean_vB.csv', 
                      dtype={
                          'q1_clean_vB': unicode,
                          'q2_clean_vB': unicode
                      } ,encoding='utf-8')


# train = pd.read_csv("data/train_clean_vB.csv",encoding="utf-8")
# test = pd.read_csv("data/test_clean_vB.csv",encoding="utf-8")

# train.fillna(unicode('empty'),inplace=True)
# test.fillna(unicode('empty'),inplace=True)

In [4]:
df_train.rename(columns={"q1_clean_vB":"question1", "q2_clean_vB":"question2"}, inplace=True)
df_test.rename(columns={"q1_clean_vB":"question1", "q2_clean_vB":"question2"}, inplace=True)

In [5]:
df_train['test_id'] = -1
df_test['id'] = -1
df_test['qid1'] = -1
df_test['qid2'] = -1
df_test['is_duplicate'] = -1

df = pd.concat([df_train, df_test])
df['question1'] = df['question1'].fillna('')
df['question2'] = df['question2'].fillna('')
df['uid'] = np.arange(df.shape[0])
df = df.set_index(['uid'])
print df.dtypes
del(df_train, df_test)

id               int64
is_duplicate     int64
qid1             int64
qid2             int64
question1       object
question2       object
test_id          int64
dtype: object


In [6]:
ix_train = np.where(df['id'] >= 0)[0]
ix_test = np.where(df['id'] == -1)[0]
ix_is_dup = np.where(df['is_duplicate'] == 1)[0]
ix_not_dup = np.where(df['is_duplicate'] == 0)[0]

In [29]:
all_questions = pd.concat((df.question1,df.question2))

In [5]:


# df1 = train[['question1']].copy()
# df2 = train[['question2']].copy()
# df1_test = test[['question1']].copy()
# df2_test = test[['question2']].copy()


# df2.rename(columns = {'question2':'question1'},inplace=True)
# df2_test.rename(columns = {'question2':'question1'},inplace=True)

# all_questions = df1.append(df2)
# all_questions = all_questions.append(df1_test)
# all_questions = all_questions.append(df2_test)

# len(all_questions) == len(df1)+len(df2)+len(df1_test)+len(df2_test)
# all_questions.shape,all_questions.drop_duplicates().shape
# all_questions = all_questions.drop_duplicates()
# sample = all_questions.iloc[0:10000]

In [34]:
import pickle

In [33]:
%%time
corpus = textacy.corpus.Corpus(lang=unicode("en"),texts=all_questions.values.tolist())

CPU times: user 1h 1min 7s, sys: 18.5 s, total: 1h 1min 25s
Wall time: 21min 49s


In [36]:
corpus.save("data/",name="textacy_all_questions_corpus")

In [73]:
# corpus = textacy.Corpus.load('data/', name='textacy_corpus')

In [46]:
# matrix_q1_train = (doc.to_terms_list(ngrams=1, named_entities=True, as_strings=True,
#                                  normalize ='lemma',filter_stops =True,filter_punct =True,filter_nums=True,min_freq=1
#                                 ) for doc in corpus)

In [37]:
doc_term_matrix, id2term = textacy.vsm.doc_term_matrix(
    (doc.to_terms_list(ngrams=1, named_entities=True, as_strings=True,
                       normalize ='lemma',filter_stops =True,filter_punct =True,filter_nums=True,min_freq=1) 
     for doc in corpus),
    weighting='tf', normalize=True, smooth_idf=True, min_df=50, max_df=0.99)

/home/ubuntu/anaconda2/envs/qenv/lib/python2.7/site-packages/sklearn/utils/validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by the normalize function.
  warnings.warn(msg, _DataConversionWarning)


In [39]:
# id2term
doc_term_matrix

<5500172x21801 sparse matrix of type '<type 'numpy.float64'>'
	with 26665690 stored elements in Compressed Sparse Row format>

In [44]:
import gc
gc.collect() 

98527223

In [ ]:
model = textacy.tm.TopicModel('nmf', n_topics=100)
model.fit(doc_term_matrix)


In [ ]:
doc_topic_matrix = model.transform(doc_term_matrix)
doc_topic_matrix.shape

In [43]:
for topic_idx, top_terms in model.top_topic_terms(id2term, top_n=2):
    print('topic', topic_idx, ':', '   '.join(top_terms))

('topic', 0, ':', u'good   place')
('topic', 1, ':', u'india   scope')
('topic', 2, ':', u'quora   answer')
('topic', 3, ':', u'difference   main')
('topic', 4, ':', u'like   look')
('topic', 5, ':', u'people   believe')
('topic', 6, ':', u'learn   python')
('topic', 7, ':', u'life   purpose')
('topic', 8, ':', u'way   easy')
('topic', 9, ':', u'know   fact')
('topic', 10, ':', u'mean   say')
('topic', 11, ':', u'work   home')
('topic', 12, ':', u'indian   economy')
('topic', 13, ':', u'english   speak')
('topic', 14, ':', u'trump   donald')
('topic', 15, ':', u'think   worry')
('topic', 16, ':', u'time   visitor')
('topic', 17, ':', u'movie   review')
('topic', 18, ':', u'happen   war')
('topic', 19, ':', u'use   us')
('topic', 20, ':', u'lose   weight')
('topic', 21, ':', u'money   earn')
('topic', 22, ':', u'find   sydney')
('topic', 23, ':', u'america   japan')
('topic', 24, ':', u'year   old')
('topic', 25, ':', u'start   preparation')
('topic', 26, ':', u'day   go')
('topic', 27,

In [50]:
model.transform(doc_term_matrix)[100]

array([  0.00000000e+00,   1.01534864e-01,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         3.71057920e-05,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         9.65896971e-03,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   1.25499014e-03,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,